In [7]:
from greenspectors.models.flair import FlairSentimentAnalyzer
from greenspectors.models.vader import VaderSentimentAnalyzer

In [16]:
flair_sentiment_analyzer = FlairSentimentAnalyzer()
vader_sentiment_analyzer = VaderSentimentAnalyzer()

sentiment_analyzers = {
    "Flair": flair_sentiment_analyzer, 
    "Vader": vader_sentiment_analyzer
}

2021-10-15 19:13:49,497 loading file /home/tobias/.flair/models/sentiment-en-mix-distillbert_4.pt


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/tobias/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
tweets_positive = [
    """Great panel discussion around the topic of #Sustainability and #netzero energy!Brilliant presentation and input from @OliviaWhitlam on #siemens sustainability strategy and journey to achieving netzero carbon.some key messages byall #collaboration #transition #trust #transparency"""
]

In [ ]:
tweets_negative = [
    """What a greenwash. Siemens are still helping Adani build a massive thermal coal mine in Australia. https://reuters.com/article/us-sie""",
    """Activists in Germany protest against new dams in the #Amazon. @Siemens: Green innovation, not Amazon destruction!""",
    """@Siemens u love green tech. and we love the Amazon. Innovation is not forest destruction.#saveTheAmazon #StopSiemens"""
]

In [17]:
for analyzer_name, sentiment_analyzer in sentiment_analyzers.items():
    print("========================")
    print(analyzer_name)
    print("========================")
    print("Positive:")
    
    for tweet in tweets_positive:
        print("\t", sentiment_analyzer.predict(tweet))
        
    print("Negative:")
    for tweet in tweets_negative:
        print("\t", sentiment_analyzer.predict(tweet))
    print()

Flair
Positive:
	 (<Sentiment.POSITIVE: 1>, 0.9973779916763306)
Negative:
	 (<Sentiment.NEGATIVE: 3>, -0.6935323476791382)
	 (<Sentiment.POSITIVE: 1>, 0.6526410579681396)
	 (<Sentiment.POSITIVE: 1>, 0.9387254118919373)

Vader
Positive:
	 (<Sentiment.POSITIVE: 1>, 0.6588)
Negative:
	 (<Sentiment.NEUTRAL: 2>, -0.1531)
	 (<Sentiment.POSITIVE: 1>, 0.5223)
	 (<Sentiment.POSITIVE: 1>, 0.9136)

